In [1]:
#omgamganapathayenamaha

In [ ]:
import boto3
import json
import os
import pickle
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
import joblib
import cloudpickle

print("Cold Start")
# grab environment variables
endpoint_name = os.environ['ENDPOINT_NAME']
client = boto3.client(service_name='sagemaker-runtime')

class sparse_to_dense:
    def fit(self, X, y=None, **fit_params):
        return self
    def transform(self, X, y=None, **fit_params):
        return X.toarray()

data_pipeline = None
if os.path.exists('/tmp/data_pipeline_downloaded') and os.path.isfile('/tmp/data_pipeline_downloaded'):
    print('Downloaded pipeline already present')
    #with open('/tmp/data_pipeline_downloaded', 'rb') as fh:
        #data_pipeline = pickle.load(fh)
    data_pipeline = joblib.load('/tmp/data_pipeline_downloaded')
else:
    print('Downloaded pipeline is not present')
    pipeln_bucket = 'ganesa-ml-sagemaker'
    pipeln_object = 'binary_text_classification/data_pipeline/data_pipeline'
    with open('/tmp/data_pipeline_downloaded', 'wb') as fw:
        boto3.Session(region_name='us-east-1').resource('s3').Bucket(pipeln_bucket).Object(pipeln_object).download_fileobj(fw)
    print('Downloaded pipeline from s3')
    #with open('/tmp/data_pipeline_downloaded', 'rb') as fh:
        #data_pipeline = pickle.load(fh)
    data_pipeline = joblib.load('/tmp/data_pipeline_downloaded')

def transform(data):
    global data_pipeline
    from __main__ import sparse_to_dense
    print("In transform")
    try:
        data_copy = json.loads(json.dumps(data))
        return data_pipeline.transform(data_copy)
    except Exception as err:
        print('Error when transforming: {0},{1}'.format(data,err))
        raise Exception('Error when transforming: {0},{1}'.format(data,err))

def lambda_handler(event, context):
    print("In lambda_handler")
    try:    
        print("Received event: " + json.dumps(event, indent=2))
        
        request = json.loads(json.dumps(event))
        
    
        X_test = [instance['features'] for instance in request['instances']]
        X_test = transform(X_test)
        print(X_test)

        result = client.invoke_endpoint(EndpointName=endpoint_name, 
                       Body='\n'.join([','.join([str(ite) for ite in item]) for item in X_test.tolist()]).encode('utf-8'),
                       ContentType='text/csv')
    
        result = result['Body'].read().decode('utf-8')
        print(result)
        y_pred = [float(item) for item in result.split() if item != ""]
        print(y_pred)
    
        # Splitting using regular expression as xgboost 1-2-2 is returning
        # predicted values with inconsistent delimiters (comma, newline or both)

        # pattern looks for one or more of non-numeric characters
        #pattern = r'[^0-9.]+'
        
        # Apply inverse transformation to get the rental count
        #result = re.split(pattern,result)
        
        return {
            'statusCode': 200,
            'isBase64Encoded':False,
            'body': json.dumps(y_pred)
        }

    except Exception as err:
        return {
            'statusCode': 400,
            'isBase64Encoded':False,
            'body': 'Call Failed {0}'.format(err)
        }